## Colab Session 연장

### 아래 코드를 F12를 누른 후 Console에 입력할것 

function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);

# CLI 환경 실행

In [21]:
!pip install kora
from kora import console
console.start()  # and click link

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
Console URL: https://teleconsole.com/s/21cb506a7b355bf756b49afbe1942d575c84307e


In [20]:
!nvidia-smi

Sun Mar  7 08:37:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    33W / 250W |  15965MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/jsk1107/dacon_k_fashion

In [ ]:
!pip install -r /content/dacon_k_fashion/requrirement.txt

# Import

In [6]:
import torch
import torch.nn as nn
from torchvision.models.detection import retinanet_resnet50_fpn
from torch.utils.data import DataLoader
from torchvision.transforms import transforms as T
from torchvision.models.detection.retinanet import RetinaNetClassificationHead
from torch.utils.data import Dataset
from pycocotools.coco import COCO
import os
import cv2
from PIL import Image
import numpy as np
from pycocotools.cocoeval import COCOeval

In [ ]:
!pip list

# DataLoader

In [33]:
class KfashionDataset(Dataset):
    def __init__(self, img_dir, anno_dir, set_name='train', transforms=None):
        self.img_dir = img_dir
        self.anno_dir = anno_dir
        self.transforms = transforms

        self.coco = COCO(os.path.join(self.anno_dir, f'{set_name}.json'))

        self.image_ids = self.coco.getImgIds()
        self.category = self.coco.loadCats(self.coco.getCatIds())

    def __getitem__(self, ids):

        img = self.get_image(ids)
        target = self.get_annotaion(ids)

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.image_ids)

    def get_image(self, ids):
        img_info = self.coco.loadImgs(self.image_ids[ids])[0]
        key = list(img_info['file_name'])[0]
        img_path = os.path.join(self.img_dir, key, img_info['file_name'])
        img = Image.open(img_path)
        # img = cv2.imread(img_path)

        return img

    def get_annotaion(self, ids):
        anno_ids = self.coco.getAnnIds(imgIds=self.image_ids[ids], iscrowd=False)
        coco_annos = self.coco.loadAnns(anno_ids)
        target = {}
        boxes = []
        labels = []
        if len(anno_ids) == 0:
            print(f'There is not annotation infomation abount Image: {self.image_ids[ids]}')
            target['boxes'] = torch.as_tensor([[10, 10, 20, 20]], dtype=torch.float32)
            target['labels'] = torch.as_tensor([[0]], dtype=torch.int64)
            return target

        for coco_anno in coco_annos:
            bbox = self.bbox_transform(coco_anno['bbox'])
            boxes.append(bbox)
            labels.append(coco_anno['category_id'])
        target['boxes'] = torch.as_tensor(boxes, dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)

        return target

    def bbox_transform(self, bbox):
        bbox[2] = bbox[0] + bbox[2]
        bbox[3] = bbox[1] + bbox[3]
        return bbox

# Train

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # (역자주: 학습시 50% 확률로 학습 영상을 좌우 반전 변환합니다)
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


img_dir = '/content/train/'
anno_dir = '/content/'
set_name = 'train'
batch_size = 16
dataset = KfashionDataset(img_dir, anno_dir, set_name, transforms=get_transform(True))
num_classes = len(dataset.category) + 1
print(f'num_classes: {num_classes}. 21+1(background)')

dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

model = retinanet_resnet50_fpn(True)
num_anchor = model.anchor_generator.num_anchors_per_location()[0]
in_channels = model.head.classification_head.cls_logits.in_channels
model.head.classification_head = RetinaNetClassificationHead(in_channels, num_anchor, num_classes)

optim = torch.optim.SGD(model.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.train()
for images, targets in dataloader:
    optim.zero_grad()

    if torch.cuda.is_available():
      model.to(device)
      images = list(image.to(device) for image in images)
      targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    out = model(images, targets)

    cls_loss = out['classification']
    reg_loss = out['bbox_regression']

    loss = cls_loss + reg_loss
    print(f'loss: {loss}')

    loss.backward()
    optim.step()